# Project pRF analysis results to pycortex surfaces
Make sure you have created the pycortex entries for these individuals (https://github.com/VisionandCognition/NHP-pycortex)     
Strategy:    
- Load all (unthresholded) results into sorted numpy arrays with nibabel
- Also create pycortex volume objects
- Save them in an oredered dictionary

In [73]:
import cortex
import nibabel as nib 
import numpy as np
import pandas as pd
import os, shutil, copy

In [37]:
subj = 'Danny' # 'Danny'/'Eddy'
FitResPath = os.path.join('/Users','chris','Documents','MRI_ANALYSIS','NHP-PRF','FitResults')

In [38]:
Models = {
    'dhrf' : ['linhrf_cv1_dhrf','linhrf_cv1_dhrf_neggain','csshrf_cv1_dhrf','doghrf_cv1_dhrf'],
    'mhrf' : ['linhrf_cv1_mhrf','linhrf_cv1_mhrf_neggain','csshrf_cv1_mhrf','doghrf_cv1_mhrf'],
    'names' : ['lin','lin_ng','css','dog'],
}
Res_type = ['ANG', 'ECC', 'EXPT', 'FWHM', 'GAIN', 'IMAG', 'REAL', 'RFS', 'X', 'Y', 'NAMP', 'SDRATIO']
xfm = 'epi2surf'

Create a dictionary of fit results for mHRF fits (all models)

In [86]:
# this will be the structure of data collection
FR = {
    'subject' : subj,
    'xfm' : xfm,
    'mHRF' : {
        'arr' : {},
        'vol' : {},
    },
    'dHRF' : {
        'arr' : {},
        'vol' : {},
    },
}

In [87]:
# Get the data
for h in ['m','d']:
    print('Processing ' + h + 'HRF')
    for M in Models[h + 'hrf']:
        # get model idx so we create a shorter variable name
        midx = Models[h + 'hrf'].index(M)
        print('Model: ' + Models['names'][midx])
        
        # get the mean R2 map
        volpath = os.path.join(FitResPath,'MRI',subj.lower(),M,'Sess-' + M + '_meanR2.nii.gz')
        volpath1 = os.path.join(FitResPath,'MRI',subj.lower(),M,'Sess-' + M + '_R2_1.nii.gz')
        volpath2 = os.path.join(FitResPath,'MRI',subj.lower(),M,'Sess-' + M + '_R2_2.nii.gz')

        # load the results into numpy arrays with nibabel
        R2 = np.array(nib.load(volpath).dataobj)
        R2_1 = np.array(nib.load(volpath1).dataobj)
        R2_2 = np.array(nib.load(volpath2).dataobj)

        # convert to pycortex volumes
        R2v = cortex.Volume(R2.transpose(2,1,0), subj, xfm)
        R2_1v = cortex.Volume(R2_1.transpose(2,1,0), subj, xfm)
        R2_2v = cortex.Volume(R2_2.transpose(2,1,0), subj, xfm)

        # add info to dictionaries
        # numpy arrays
        FitRes = {
            'R2' : R2,
            'R2_1' : R2_1,
            'R2_2' : R2_2
            }
        # pycortex volumes
        FitRes_vol = {
            'R2' : R2v,
            'R2_1' : R2_1v,
            'R2_2' : R2_2v
            }

        # also get othere results
        for R in Res_type:
            volpath = os.path.join(FitResPath,'MRI',subj.lower(),M,'TH_0', R + '_th0.nii.gz')
            if os.path.exists(volpath):
                FitRes[R] = np.array(nib.load(volpath).dataobj)
                FitRes_vol[R] = cortex.Volume(FitRes[R].transpose(2,1,0), subj, xfm)

        # bring it all together
        FR[h + 'HRF']['arr'][Model_names[midx]] = FitRes
        FR[h + 'HRF']['vol'][Model_names[midx]] = FitRes_vol

Processing mHRF
Model: lin
Model: lin_ng
Model: css
Model: dog
Processing dHRF
Model: lin
Model: lin_ng
Model: css
Model: dog


Check in webviewer

In [69]:
cortex.webgl.show(data=FR['mHRF']['vol']['lin'])

Started server on port 60281


<JS: window.viewer>

Stopping server


In [76]:
# Inspect the result volumes (should be numpy arrays)
RR = copy.copy(FR['mHRF']['arr']['lin']['R2'])
RR[RR < 5] ='nan'
cortex.webgl.show(data=cortex.Volume(RR.transpose(2,1,0), subj, xfm))

Define a function to mask the results based on R2 values

In [93]:
def r2mask(DataDict,RTH):
    # mask the numpy arrays by inserting nan's
    for hrf in ['mHRF','dHRF']:
        for m in DataDict[hrf]['arr']:
            for res in DataDict[hrf]['arr'][m]:
                if res is not 'R2':
                    DataDict[hrf]['arr'][m][res][ DataDict[hrf]['arr'][m]['R2'] < RTH ] = 'nan'
                    # also convert to the pycortex volume
                    DataDict[hrf]['vol'][m][res] = cortex.Volume(DataDict[hrf]['arr'][m][res].transpose(2,1,0), DataDict['subject'], DataDict['xfm'])
                        
            DataDict[hrf]['arr'][m]['R2'][ DataDict[hrf]['arr'][m]['R2'] < RTH ] = 'nan' 
            DataDict[hrf]['vol'][m]['R2'] = cortex.Volume(DataDict[hrf]['arr'][m]['R2'].transpose(2,1,0), DataDict['subject'], DataDict['xfm'])

    return DataDict

In [94]:
mFR = copy.deepcopy(FR)
mFR = r2mask(mFR,2)
cortex.webgl.show(data=mFR['mHRF']['vol']['lin'])

Started server on port 34261


<JS: window.viewer>

Stopping server
